In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


#import matplotlib.pyplot as plt
#import numpy as np
import pandas as pd
#import networkx as nx
#import csv
#from sys import stdin
#from networkx.algorithms.community import greedy_modularity_communities
#from networkx.algorithms.community import k_clique_communities
#from networkx.algorithms.community import asyn_lpa_communities
#from networkx.algorithms.community import modularity
#from cdlib import algorithms, readwrite, viz
#from sklearn.manifold import TSNE
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.metrics import accuracy_score
#from gensim.models import Word2Vec
from sklearn import preprocessing
import sklearn.metrics as sk_metrics
#import seaborn as sns

#################
def fixed_metrics(y,y_pred):
    from sklearn import metrics
    import pandas as pd
    import sklearn.metrics as sk_metrics
    

    cm = sk_metrics.confusion_matrix(y, y_pred)

    results_partial = y.copy()
    results_partial=pd.DataFrame(results_partial)
    results_partial["pred"] = y_pred
    results_partial = results_partial[(results_partial.iloc[:,0]<1)]
    
    alfa = sum(results_partial["pred"])/results_partial.shape[0]
    beta = 1
    
    fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=1)

    
    TN = cm[0,0]
    FN = cm[1,0]
    TP = cm[1,1]
    FP = cm[0,1]
    
    theta = (TP+FP)/(TP+FN+TN+FP)
    n=FP/(TN+FP)
    #pi = (TP+FN)/(TP+FN+TN+FP)
    c= len(y[(y.iloc[:,0]==1)])/(len(y[(y.iloc[:,0]==1)])+len(y[(y.iloc[:,0]==0)]))
    gamma = TP/(TP+FN)
    if beta-alfa==0:
        gamma_cr = float('inf')
        n_cr = float('inf')
    else:
        gamma_cr = ((beta-alfa)**-1)*((1-alfa)*gamma-(1-beta)*n)
        n_cr = ((beta-alfa)**-1)*((beta*n)-(alfa*gamma))
        
    pi_cr = (c*beta)+((1-c)*alfa)
    ##metrics
    ACC_cr = (pi_cr*gamma_cr) + ((1-pi_cr)*(1-n_cr))
    bacc_cr = (1 + (gamma_cr - n_cr))/2
    f_cr = (2*pi_cr*gamma_cr)/(pi_cr+theta)
    mcc_cr = (pi_cr*(1-pi_cr)/theta*(1-theta))*(gamma_cr-n_cr)
    
    auc_r = metrics.auc(fpr, tpr)
    auc_cr = (auc_r-(1-(beta-alfa)))/(beta-alfa)
    alfa = sum(results_partial["pred"])/results_partial.shape[0]
    
    metrics_df =[[auc_r,auc_cr,ACC_cr,bacc_cr,f_cr,mcc_cr,alfa]]
    return(metrics_df)


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
filename = "D:/TESIS_PHD/RICENETPPI/out_pred_JJG.csv"
#filename = "/users/ccsosaa/pecanpy/BIG_COMP_W.emb"
data = pd.read_csv(filename,index_col=0)
#print(data)

# PROB >= 0.9

In [3]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_svm"]
y_pred = y_pred.to_frame()
fixed_SVM = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_SVM)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.5, 0.5, 0.9964012073368934, 0.5, 0.0, nan, 0.0]]


C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3845178042.py:69: RuntimeWarning: divide by zero encountered in double_scalars
  mcc_cr = (pi_cr*(1-pi_cr)/theta*(1-theta))*(gamma_cr-n_cr)
C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3845178042.py:69: RuntimeWarning: invalid value encountered in double_scalars
  mcc_cr = (pi_cr*(1-pi_cr)/theta*(1-theta))*(gamma_cr-n_cr)


In [4]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_lda"]
y_pred = y_pred.to_frame()
fixed_LDA = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_LDA)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.9919354838709677, 0.9919354838709677, 0.9999419549570466, 0.9919354838709677, 0.991869918699187, 0.9928732021332975, 0.0]]


In [5]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_rf"]
y_pred = y_pred.to_frame()
fixed_RF = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_RF)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.5, 0.5, 0.9964012073368934, 0.5, 0.0, nan, 0.0]]


C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3845178042.py:69: RuntimeWarning: divide by zero encountered in double_scalars
  mcc_cr = (pi_cr*(1-pi_cr)/theta*(1-theta))*(gamma_cr-n_cr)
C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3845178042.py:69: RuntimeWarning: invalid value encountered in double_scalars
  mcc_cr = (pi_cr*(1-pi_cr)/theta*(1-theta))*(gamma_cr-n_cr)


# PROB >= 0.8

In [6]:
for i in range(len(data["pred_svm"])):
    if data["prob_svm"][i] >= 0.8:
        data["pred_svm"][i] = 1
    else:
        data["pred_svm"][i] = 0

for i in range(len(data["pred_lda"])):
    if data["prob_lda"][i] >= 0.8:
        data["pred_lda"][i] = 1
    else:
        data["pred_lda"][i] = 0

for i in range(len(data["pred_rf"])):
    if data["prob_rf"][i] >= 0.8:
        data["pred_rf"][i] = 1
    else:
        data["pred_rf"][i] = 0


C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3509507299.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["pred_svm"][i] = 0
C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3509507299.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["pred_lda"][i] = 0
C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3509507299.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["pred_lda"][i] = 1
C:\Users\cami_\AppData\Local\T

In [7]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_svm"]
y_pred = y_pred.to_frame()
fixed_SVM = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_SVM)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.5, 0.5, 0.9964012073368934, 0.5, 0.0, nan, 0.0]]


C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3845178042.py:69: RuntimeWarning: divide by zero encountered in double_scalars
  mcc_cr = (pi_cr*(1-pi_cr)/theta*(1-theta))*(gamma_cr-n_cr)
C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\3845178042.py:69: RuntimeWarning: invalid value encountered in double_scalars
  mcc_cr = (pi_cr*(1-pi_cr)/theta*(1-theta))*(gamma_cr-n_cr)


In [8]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_lda"]
y_pred = y_pred.to_frame()
fixed_LDA = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_LDA)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.9998834906209949, 0.9998834634657965, 1.0, 1.0, 1.0, 0.9923527306830222, 0.0002330187580100198]]


In [9]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_rf"]
y_pred = y_pred.to_frame()
fixed_RF = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_RF)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.8548387096774194, 0.8548387096774194, 0.99895518922684, 0.8548387096774194, 0.8301886792452831, 0.9938564166982341, 0.0]]


# PROB >= 0.7

In [10]:
for i in range(len(data["pred_svm"])):
    if data["prob_svm"][i] >= 0.7:
        data["pred_svm"][i] = 1
    else:
        data["pred_svm"][i] = 0

for i in range(len(data["pred_lda"])):
    if data["prob_lda"][i] >= 0.7:
        data["pred_lda"][i] = 1
    else:
        data["pred_lda"][i] = 0

for i in range(len(data["pred_rf"])):
    if data["prob_rf"][i] >= 0.7:
        data["pred_rf"][i] = 1
    else:
        data["pred_rf"][i] = 0


C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\2643906724.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["pred_svm"][i] = 0
C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\2643906724.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["pred_lda"][i] = 0
C:\Users\cami_\AppData\Local\Temp\ipykernel_9476\2643906724.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["pred_lda"][i] = 1
C:\Users\cami_\AppData\Local\T

In [11]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_A_svm"]
y_pred = y_pred.to_frame()
fixed_SVM = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_SVM)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.006582779913783032, -74.4557522123897, -0.9701648479219841, -36.97787610619485, 0.0, -0.016497676122837996, 0.9868344401724339]]


In [12]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_A_lda"]
y_pred = y_pred.to_frame()
fixed_LDA = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_LDA)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.7641558895491087, 0.5535891498511414, 1.0, 1.0, 1.0, 0.27710800876581226, 0.4716882209017826]]


In [13]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_A_rf"]
y_pred = y_pred.to_frame()
fixed_RF = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_RF)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.7217756029360364, 0.37273443656422395, 0.9999999999999999, 1.0, 0.9999999999999998, 0.19532418405812232, 0.5564487941279272]]


# PROB max between 0 and 1 classes

In [14]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_A_svm"]
y_pred = y_pred.to_frame()
fixed_SVM = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_SVM)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.006582779913783032, -74.4557522123897, -0.9701648479219841, -36.97787610619485, 0.0, -0.016497676122837996, 0.9868344401724339]]


In [15]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_A_lda"]
y_pred = y_pred.to_frame()
fixed_LDA = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_LDA)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.7641558895491087, 0.5535891498511414, 1.0, 1.0, 1.0, 0.27710800876581226, 0.4716882209017826]]


In [16]:
y = data["1"]
y = y.to_frame()
y_pred = data["pred_A_rf"]
y_pred = y_pred.to_frame()
fixed_RF = fixed_metrics(y,y_pred)
print("AUC","AUCcr","ACCURACY","B_ACCURACY","F","MCC","ALPHA")
print(fixed_RF)

AUC AUCcr ACCURACY B_ACCURACY F MCC ALPHA
[[0.7217756029360364, 0.37273443656422395, 0.9999999999999999, 1.0, 0.9999999999999998, 0.19532418405812232, 0.5564487941279272]]
